<a href="https://colab.research.google.com/github/Madhavinders/Business_analytics_project/blob/main/BusinessAnalytics_Parking_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Model 1
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("dataset.csv")

# STEP 1: DATA PREPROCESSING

# Combine date and time into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Encode traffic levels
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

# Encode vehicle types with weights
vehicle_type_weights = {'car': 1.0, 'bike': 0.6, 'truck': 1.5, 'cycle': 0.3}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_type_weights)

# Compute occupancy rate
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']

# Sort by lot and timestamp to simulate streaming
df.sort_values(by=['SystemCodeNumber', 'Timestamp'], inplace=True)
df.reset_index(drop=True, inplace=True)

# STEP 2: MODEL 1 — BASELINE PRICING

BASE_PRICE = 10.0  # Starting price
ALPHA = 2.0        # Sensitivity to occupancy

# Initialize column to store predicted prices
df['Price_Model1'] = np.nan

# Dictionary to track last known price per parking lot
last_price_by_lot = {}

# Iterate over each row to simulate real-time pricing
for idx, row in df.iterrows():
    lot = row['SystemCodeNumber']
    occupancy_rate = row['OccupancyRate']

    if lot not in last_price_by_lot:
        price = BASE_PRICE  # Initial price
    else:
        price = last_price_by_lot[lot] + ALPHA * occupancy_rate

    df.at[idx, 'Price_Model1'] = price
    last_price_by_lot[lot] = price  # Update tracker

# Preview results of first 10 rows
print(df[['SystemCodeNumber', 'Timestamp', 'OccupancyRate', 'Price_Model1']].head(10))


  SystemCodeNumber           Timestamp  OccupancyRate  Price_Model1
0      BHMBCCMKT01 2016-10-04 07:59:00       0.105719     10.000000
1      BHMBCCMKT01 2016-10-04 08:25:00       0.110919     10.221837
2      BHMBCCMKT01 2016-10-04 08:59:00       0.138648     10.499133
3      BHMBCCMKT01 2016-10-04 09:32:00       0.185442     10.870017
4      BHMBCCMKT01 2016-10-04 09:59:00       0.259965     11.389948
5      BHMBCCMKT01 2016-10-04 10:26:00       0.306759     12.003466
6      BHMBCCMKT01 2016-10-04 10:59:00       0.379549     12.762565
7      BHMBCCMKT01 2016-10-04 11:25:00       0.428076     13.618718
8      BHMBCCMKT01 2016-10-04 11:59:00       0.448873     14.516464
9      BHMBCCMKT01 2016-10-04 12:29:00       0.461005     15.438475


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column
from bokeh.io import push_notebook

output_notebook()

# Filter relevant data
plot_df_1 = df[['SystemCodeNumber', 'Timestamp', 'Price_Model1']].copy()
plot_df_1['Timestamp'] = pd.to_datetime(plot_df_1['Timestamp'])

# Initial lot
initial_lot = plot_df_1['SystemCodeNumber'].unique()[0]
lot_df = plot_df_1[plot_df_1['SystemCodeNumber'] == initial_lot]

source = ColumnDataSource(data={
    'x': lot_df['Timestamp'],
    'y': lot_df['Price_Model1']
})

# Create the figure
p = figure(
    x_axis_type='datetime',
    width=800,
    height=400,
    title=f"Model 1 Pricing — Lot: {initial_lot}",
    y_axis_label='Price ($)',
    x_axis_label='Time'
)

line = p.line('x', 'y', source=source, line_width=2, color="green")

# Dropdown menu
lot_select = Select(title="Select Parking Lot:", value=initial_lot,
                    options=list(plot_df_1['SystemCodeNumber'].unique()))

# Callback to update data
def update_plot(attr, old, new):
    selected_lot = lot_select.value
    new_df = plot_df_1[plot_df_1['SystemCodeNumber'] == selected_lot]
    source.data = {
        'x': new_df['Timestamp'],
        'y': new_df['Price_Model1']
    }
    p.title.text = f"Model 1 Pricing — Lot: {selected_lot}"
    push_notebook()

lot_select.on_change('value', update_plot)

# Show
show(column(lot_select, p), notebook_handle=True)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/js_callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [ ]:
# Export Model 1 Dashboard as HTML
from bokeh.plotting import figure, output_file, save
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Select

# Rebuild source and figure (fresh, not shared with notebook)
lot_df_export = plot_df_1[plot_df_1['SystemCodeNumber'] == initial_lot]
export_source = ColumnDataSource(data={
    'x': lot_df_export['Timestamp'],
    'y': lot_df_export['Price_Model1']
})

export_fig = figure(
    x_axis_type='datetime',
    width=800,
    height=400,
    title=f"Model 1 Pricing — Lot: {initial_lot}",
    y_axis_label='Price ($)',
    x_axis_label='Time'
)
export_fig.line('x', 'y', source=export_source, line_width=2, color="green")

output_file("model1_dashboard.html", title="Model 1 Pricing Dashboard")
save(export_fig)

# Download in Colab
from google.colab import files
files.download("model1_dashboard.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Model 2
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("dataset.csv")

# STEP 1: DATA PREPROCESSING

# Combine date and time into timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Encode traffic levels
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

# Encode vehicle type weights
vehicle_type_weights = {'car': 1.0, 'bike': 0.6, 'truck': 1.5, 'cycle': 0.3}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_type_weights)

# Calculate occupancy rate
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']

# Sort for simulation
df.sort_values(by=['SystemCodeNumber', 'Timestamp'], inplace=True)
df.reset_index(drop=True, inplace=True)

# STEP 2: DEMAND FUNCTION CALCULATION

# Coefficients for each component
alpha = 2.0     # Occupancy
beta = 0.5      # Queue length
gamma = 1.0     # Traffic (penalty)
delta = 2.0     # Special day boost
epsilon = 1.0   # Vehicle weight
lambda_ = 0.8   # Scaling demand to price

# Base price
BASE_PRICE = 10.0

# Calculate raw demand score
df['RawDemand'] = (
    alpha * df['OccupancyRate'] +
    beta * df['QueueLength'] -
    gamma * df['TrafficLevel'] +
    delta * df['IsSpecialDay'] +
    epsilon * df['VehicleWeight']
)

# Normalize demand within each parking lot for each day
df['Date'] = df['Timestamp'].dt.date
df['LotDayKey'] = df['SystemCodeNumber'].astype(str) + '_' + df['Date'].astype(str)

# Min-max normalization group-wise
demand_min = df.groupby('LotDayKey')['RawDemand'].transform('min')
demand_max = df.groupby('LotDayKey')['RawDemand'].transform('max')
df['NormalizedDemand'] = (df['RawDemand'] - demand_min) / (demand_max - demand_min + 1e-6)

# STEP 3: CALCULATE DYNAMIC PRICE

# Price = base × (1 + lambda × normalized demand)
df['Price_Model2'] = BASE_PRICE * (1 + lambda_ * df['NormalizedDemand'])

# Clip prices between 0.5x and 2x base
df['Price_Model2'] = df['Price_Model2'].clip(lower=BASE_PRICE * 0.5, upper=BASE_PRICE * 2.0)

# Preview results
print(df[['SystemCodeNumber', 'Timestamp', 'RawDemand', 'NormalizedDemand', 'Price_Model2']].head(10))


  SystemCodeNumber           Timestamp  RawDemand  NormalizedDemand  \
0      BHMBCCMKT01 2016-10-04 07:59:00   0.711438          0.106483   
1      BHMBCCMKT01 2016-10-04 08:25:00   0.721837          0.111615   
2      BHMBCCMKT01 2016-10-04 08:59:00   1.277296          0.385734   
3      BHMBCCMKT01 2016-10-04 09:32:00   1.370884          0.431919   
4      BHMBCCMKT01 2016-10-04 09:59:00   1.119931          0.308074   
5      BHMBCCMKT01 2016-10-04 10:26:00   2.113518          0.798409   
6      BHMBCCMKT01 2016-10-04 10:59:00   2.259099          0.870253   
7      BHMBCCMKT01 2016-10-04 11:25:00   2.356153          0.918149   
8      BHMBCCMKT01 2016-10-04 11:59:00   1.697747          0.593226   
9      BHMBCCMKT01 2016-10-04 12:29:00   2.522010          1.000000   

   Price_Model2  
0     10.851864  
1     10.892918  
2     13.085869  
3     13.455352  
4     12.464590  
5     16.387270  
6     16.962022  
7     17.345190  
8     14.745807  
9     17.999996  


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column
from bokeh.io import push_notebook

output_notebook()

# Filter relevant columns
plot_df = df[['SystemCodeNumber', 'Timestamp', 'Price_Model2']].copy()
plot_df['Timestamp'] = pd.to_datetime(plot_df['Timestamp'])

# Create initial data source (first lot by default)
initial_lot = plot_df['SystemCodeNumber'].unique()[0]
lot_df = plot_df[plot_df['SystemCodeNumber'] == initial_lot]

source = ColumnDataSource(data={
    'x': lot_df['Timestamp'],
    'y': lot_df['Price_Model2']
})

# Create the plot
p = figure( x_axis_type='datetime',width=800, height=400,
    title=f"Dynamic Pricing Over Time — Lot: {initial_lot}",
    y_axis_label='Price ($)',
    x_axis_label='Time'
)

line = p.line('x', 'y', source=source, line_width=2, color="navy")

# Create dropdown menu
lot_select = Select(title="Select Parking Lot:", value=initial_lot,
                    options=list(plot_df['SystemCodeNumber'].unique()))

# Update function
def update_plot(attr, old, new):
    selected_lot = lot_select.value
    new_df = plot_df[plot_df['SystemCodeNumber'] == selected_lot]
    source.data = {
        'x': new_df['Timestamp'],
        'y': new_df['Price_Model2']
    }
    p.title.text = f"Dynamic Pricing Over Time — Lot: {selected_lot}"
    push_notebook()

lot_select.on_change('value', update_plot)

# Dashboard
show(column(lot_select, p), notebook_handle=True)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/js_callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [ ]:
# Export Model 2 Dashboard as HTML
from bokeh.plotting import figure, output_file, save
from bokeh.layouts import column
from bokeh.models import ColumnDataSource

# Rebuild fresh data source for export
lot_df_export_2 = plot_df[plot_df['SystemCodeNumber'] == initial_lot]
export_source_2 = ColumnDataSource(data={
    'x': lot_df_export_2['Timestamp'],
    'y': lot_df_export_2['Price_Model2']
})

# Create new figure for export
export_fig_2 = figure(
    x_axis_type='datetime',
    width=800,
    height=400,
    title=f"Model 2 Pricing — Lot: {initial_lot}",
    y_axis_label='Price ($)',
    x_axis_label='Time'
)

export_fig_2.line('x', 'y', source=export_source_2, line_width=2, color="navy")

# Save as HTML
output_file("model2_dashboard.html", title="Model 2 Pricing Dashboard")
save(export_fig_2)

# Download it in Colab
from google.colab import files
files.download("model2_dashboard.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>